In [ ]:
#import libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot

In [ ]:
#import algorithms specifically from scikit- learn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [ ]:
#import daset
df = pd.read_csv("AmesHousing.csv")
headers = df.iloc[0] #stores headers

objects = df.select_dtypes(include=['object']).columns #selects all columns with non-numeric data
df = df.drop(columns = objects, axis = 1) #gets rid of all columns with non-numeric data
df

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
0,1,526301100,20,141.0,31770,6,5,1960,1960,112.0,...,210,62,0,0,0,0,0,5,2010,215000
1,2,526350040,20,80.0,11622,5,6,1961,1961,0.0,...,140,0,0,0,120,0,0,6,2010,105000
2,3,526351010,20,81.0,14267,6,6,1958,1958,108.0,...,393,36,0,0,0,0,12500,6,2010,172000
3,4,526353030,20,93.0,11160,7,5,1968,1968,0.0,...,0,0,0,0,0,0,0,4,2010,244000
4,5,527105010,60,74.0,13830,5,5,1997,1998,0.0,...,212,34,0,0,0,0,0,3,2010,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,37.0,7937,6,6,1984,1984,0.0,...,120,0,0,0,0,0,0,3,2006,142500
2926,2927,923276100,20,NaN,8885,5,5,1983,1983,0.0,...,164,0,0,0,0,0,0,6,2006,131000
2927,2928,923400125,85,62.0,10441,5,5,1992,1992,0.0,...,80,32,0,0,0,0,700,7,2006,132000
2928,2929,924100070,20,77.0,10010,5,5,1974,1975,0.0,...,240,38,0,0,0,0,0,4,2006,170000


In [ ]:
#get rid of headers
j = 0
for i in df.columns:
  df = df.rename(columns={i: j})
  j+=1
df = df.drop(axis = 1, columns = 0) #get rid of extra indexs per row

In [ ]:
df

,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
0,526301100,20,141.0,31770,6,5,1960,1960,112.0,639.0,...,210,62,0,0,0,0,0,5,2010,215000
1,526350040,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,140,0,0,0,120,0,0,6,2010,105000
2,526351010,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,393,36,0,0,0,0,12500,6,2010,172000
3,526353030,20,93.0,11160,7,5,1968,1968,0.0,1065.0,...,0,0,0,0,0,0,0,4,2010,244000
4,527105010,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,212,34,0,0,0,0,0,3,2010,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,923275080,80,37.0,7937,6,6,1984,1984,0.0,819.0,...,120,0,0,0,0,0,0,3,2006,142500
2926,923276100,20,NaN,8885,5,5,1983,1983,0.0,301.0,...,164,0,0,0,0,0,0,6,2006,131000
2927,923400125,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,80,32,0,0,0,0,700,7,2006,132000
2928,924100070,20,77.0,10010,5,5,1974,1975,0.0,1071.0,...,240,38,0,0,0,0,0,4,2006,170000


In [ ]:
#find missing values and fill them with average of columns
df.isnull().sum().sum()
df[1].mean()
for column in df:
  df[column].fillna(value=df[column].mean(), inplace=True)


In [ ]:
#split data into features (features = x) and labels (labels = y)
x = df.loc[:,1:38]
y = df.loc[:,38]


In [ ]:
#scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
#split features and labels into traning and testing data (80% of data for training, 20% for testing)
from sklearn.model_selection import train_test_split
#split data into train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#create model and fit training data through model
model = LinearRegression()
model.fit(x_train,y_train)
y_model = model.predict(x_test)
from sklearn.metrics import mean_squared_error, r2_score
print('Mean Squared Error:', mean_squared_error(y_test, y_model))
print('R^2 Score:', r2_score(y_test, y_model))

Mean Squared Error: 8.263268944054644e-21
R^2 Score: 1.0


In [ ]:
#svm regression model with hypertuning
model = SVR(C=100, gamma=1, kernel='linear')
model.fit(x_train,y_train)
#run test values through model
y_model = model.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_model))
print('R^2 Score:', r2_score(y_test, y_model))

Mean Squared Error: 97508074.89483793
R^2 Score: 0.9878381675126702


In [ ]:
#knn regression model with no hypertuning
model = KNeighborsRegressor(n_neighbors=7)
model.fit(x_train,y_train)
#run test values through model
x_test = x_test.astype("float16")
y_model = model.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_model))
print('R^2 Score:', r2_score(y_test, y_model))

Mean Squared Error: 922374247.6006825
R^2 Score: 0.8849555680178828


In [ ]:
#Random Forest Regressor model with no hypertuning
model = RandomForestRegressor()
model.fit(x_train,y_train)
#run test values through model
y_model = model.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_model))
print('R^2 Score:', r2_score(y_test, y_model))

Mean Squared Error: 6962047.907099147
R^2 Score: 0.9991316487326181


In [ ]:
#Gradient Boost model with no hypertuning
model = RandomForestRegressor()
model.fit(x_train,y_train)
#run test values through model
y_model = model.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_model))
print('R^2 Score:', r2_score(y_test, y_model))

Mean Squared Error: 2457659.8149117813
R^2 Score: 0.999693464905219
